# Aligning Letters

In [1]:
import json
import pytrie
from itertools import permutations
import re
import random
import requests
import tempfile

In [2]:
with open("./out/letters_matrix.json", 'r') as jf:
    letters_matrix = json.load(jf)

## Find the 4 starting columns of the letter
**Filter:** The first 4 columns of the letter have to have no space or punctuation character within them. That is based on the hint that Dr Dexter always makes sure that every line of any letter he write starts with at words that are at least 4 letters long.

In [3]:
def is_likely_a_starting_column(col):
    for char in col:
        if char in [" ", ".", ","]:
            return False
    return True

In [4]:
def find_all_likely_starting_columns_indices(letters_matrix=letters_matrix):
    indices = set()
    for idx, col in enumerate(letters_matrix):
        if is_likely_a_starting_column(col):
            indices.add(idx)
    return indices

In [5]:
scols_indices = find_all_likely_starting_columns_indices()

In [6]:
scols_indices

{26, 47, 54, 69}

## Finding the right permutation of the starting columns
**Trick:** The 4 starting columns have to all be a valid english word prefix.

In [7]:
#english_words_file = 'words_alpha.txt'

In [8]:
def load_words_alpha_corpus():
    url = "https://github.com/dwyl/english-words/raw/refs/heads/master/words_alpha.txt"

    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Ensure successful download
        with tempfile.NamedTemporaryFile(delete=False, mode='w+', encoding='utf-8') as tmp_file:
            for chunk in response.iter_content(chunk_size=8192):
                tmp_file.write(chunk.decode())  # Decode bytes to string
            
            english_words_file = tmp_file.name  # Save file path before closing

    # Load corpus directly
    with open(english_words_file, 'r', encoding='utf-8') as file:
        wordlist = [line.strip() for line in file.readlines()]
    
    return wordlist

words_alpha_corpus = load_words_alpha_corpus()

def build_trie(corpus=words_alpha_corpus):
    trie = pytrie.Trie()
    for word in corpus:
        trie[word] = True
    return trie

# Build the trie index
trie = build_trie()

_prefix_cach = {}

def prefix_match(prefix, lower=True, trie=trie):
    if lower:
        prefix = prefix.lower()
    if prefix in _prefix_cach:
        return _prefix_cach[prefix]
    else:
        prefix_tuple = tuple(prefix)
        _prefix_cach[prefix] = set(["".join(word) for word in trie.keys(prefix_tuple) if ''.join(word).startswith(prefix)])
        return _prefix_cach[prefix]

def prefix_is_valid(prefix, trie=trie):
    return len(prefix_match(prefix, trie=trie)) > 0

In [9]:
def all_prefixes_are_valid(prefixes, print_invalid_prefix=False):
    if len(prefixes) == 0:
        raise Exception("Empty prefixes")
    for p in prefixes:
        if not prefix_is_valid(p):
            if print_invalid_prefix:
                print(f"INVALID_PREFIX: {p}")
            return False
    return True

In [10]:
def identify_starting_permutation(likely_scols_indices=scols_indices):
    starting_perms = []
    # All possible permutations of length 4 for the starting columns indices
    perms = list(permutations(likely_scols_indices, 4))

    # Evaluate each permutation: if all words in the colum are valid prefixes, we retain the permutation
    for perm in perms:
        prefixes = ["" for _ in range(32)]
        for colidx in perm:
            for i in range(len(letters_matrix[0])):
                prefixes[i] += letters_matrix[colidx][i]
        if all_prefixes_are_valid(prefixes):
            starting_perms.append(perm)
    return starting_perms

In [11]:
scols_indices

{26, 47, 54, 69}

In [12]:
starting_permutations = identify_starting_permutation(scols_indices)

In [13]:
starting_permutations

[(54, 26, 69, 47)]

In [14]:
if(len(starting_permutations) > 1):
    raise Exception("More than one valid permutation for starting columns was detected")

## Rebuilding the rest of the columns

In [15]:
sorted_cols = list(starting_permutations[0])

In [16]:
sorted_cols

[54, 26, 69, 47]

In [17]:
all_remaining_cols_indices = [idx for idx, _ in enumerate(letters_matrix) if idx not in sorted_cols]

In [18]:
def split_into_words(strings):
    words = set()
    for s in strings:
        words.update(re.split(r'[ .,]+', s))  # Split on space, dot, or comma
    words.discard('')  # Remove empty strings if any
    return words

In [19]:
def sort_by_column_spaces(int_array, char_matrix):
    return sorted(int_array, key=lambda i: sum(char_matrix[i][idx] in [' '] for idx in range(len(char_matrix[0]))))

In [20]:
def solve(starting_indices, remaining_indices):
    if len(remaining_indices) == 0:
        return (True, starting_indices)
    possible_next = []
    random.shuffle(remaining_indices)
    for idx in remaining_indices:
        rows = ["" for _ in range(32)]
        for sidx in starting_indices:
            for i in range(len(letters_matrix[0])):
                rows[i] += letters_matrix[sidx][i]
                
        for i in range(len(letters_matrix[0])):
            rows[i] += letters_matrix[idx][i]
            
        
        if all(prefix_is_valid(w) for w in split_into_words(rows)):
            possible_next.append(idx)
            
    if len(possible_next) == 0:
        return (False, None)
        
    # print("Possible next first row: " +( ",".join([letters_matrix[p][0] for p in possible_next])))
    # print("---")

    possible_next = sort_by_column_spaces(possible_next, letters_matrix)
    for pn in possible_next:
        next_starting_indices = starting_indices.copy()
        next_starting_indices.append(pn)

        # print(next_starting_indices)

        if random.randint(1, 1024) == 1:
            # Printing the first two lines for debugging
            for line in range(5):
                print("".join(letters_matrix[nidx][line] for nidx in next_starting_indices))
            print("---")

        next_remaining_indices = remaining_indices.copy()
        next_remaining_indices.remove(pn)

        out = solve(next_starting_indices, next_remaining_indices)
        if out[0] == True:
            return out
    return (False, None)

In [21]:
results = solve(sorted_cols, all_remaining_cols_indices)

# Dr. DEXTERS'S LETTER

In [22]:
def matrix_to_text(seq, m):
    txt = ""
    for y in range(len(m[0])):
        for idx in seq:
            txt += m[idx][y]
        txt+="\n"
    return txt

In [23]:
reconstructed_letter = matrix_to_text(results[1], letters_matrix)

In [24]:
print(reconstructed_letter)

DEAR STUDENT OF MINE                                                                                
UNCOMMON AS IT IS FOR SOMEONE LIKE ME TO ADMIT ANY FAULT  BUT I FIND MYSELF IN A PECULIAR POSITION  
ALAS. THERE WAS A TIME  NOT SO LONG AGO  WHEN I LOOKED AT YOUR WORK WITH DISDAIN  WHEN I SAW IT AS  
LITTLE MORE THAN A MISGUIDED  FRIVOLOUS EXPERIMENT WITH NO REAL MERIT OR VALUE. IN MY ARROGANCE  I  
THOUGHT YOUR PROJECT WAS DESTINED TO FAIL  A FUTILE EXERCISE IN FUTILITY. PERHAPS  I WAS TOO QUICK  
DISMISSING IT  TOO BLINDED BY MY OWN CONFIDENCE IN MY UNDERSTANDING OF THESE MATTERS. PROTONS, OR   
PROTEINS  THE VERY CORE OF THE IDEAS YOU WERE GRAPPLING WITH  I THOUGHT WERE BEYOND YOUR REACH.     
CERTAINLY  YOU LACKED THE SKILL TO MAKE ANY SENSE OF THESE COMPLEX CONCEPTS  LET ALONE MASTER THEM. 
HOWEVER  IN HINDSIGHT  I SEE I WAS MISTAKEN. HERE WE ARE  AND I MUST ADMIT  AGAINST ALL MY          
EXPECTATIONS  YOU HAVE DONE IT. THE RESULTS ARE IMPRESSIVE  IN THEIR OWN RIGHT  THOUGH I AM

In [25]:
with open("./out/reconstructed_letter.txt", 'w') as f:
    f.write(reconstructed_letter)